In [1]:
import json
import pika
import uuid
import threading 
import datetime
import time
import logging
import random
import requests
import os
from enum import Enum
from flask import Flask, request, jsonify
from flask_api import status
from pymessenger import Element, Button
from requests_toolbelt import MultipartEncoder
from pymessenger import utils
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
class NotificationType(Enum):
    regular = "REGULAR"
    silent_push = "SILENT_PUSH"
    no_push = "NO_PUSH"

In [3]:
def verify_fb_token(token_sent,exito):
    VERIFY_TOKEN='chatbot' # token de verificación para fb  
    if token_sent == VERIFY_TOKEN:
        return exito
    else:
        return 'Verificacion de token invalida'

In [4]:
def enviarNucleoLectura(recipient_id, id_face, mensaje,hilo): 
    content=''   
    entidad={}
    n =  {"payload": {"robot":"none","id_face":" ","recipient_id":" ","access_token":"none","content":" ","type_resp":"read","state":"none", "blocktype":"none", "opciones":"none", "entities":""}}
    
    data=json.dumps(n)
    decoded=json.loads(data)
    decoded["payload"]["id_face"]=id_face
    decoded["payload"]["recipient_id"]=recipient_id
    decoded["payload"]["content"]=content
    decoded["payload"]["entities"]= entidad  
    
    data_string=json.dumps(decoded)    
    hilo.corr_id = str(uuid.uuid4())
    hilo.channel.basic_consume(on_message_callback=hilo.on_response, auto_ack=True, queue=hilo.queue_name) 
    try:
        hilo.channel.basic_publish(exchange='',
                              routing_key='Colaprueba23',
                              properties=pika.BasicProperties(
                                        reply_to = hilo.queue_name,
                                        correlation_id = hilo.corr_id,
                                        ),
                                body=str(data_string))
               
        #print('\n________________________________________read _______________________________________________________________________________')
        #print('Mensaje enviado al Nucleo Logico Read-> ',data_string)  
        hilo.start() 
    except Exception as error:
        print("mensaje no enviado...",str(error))

In [5]:
def enviarNucleo(recipient_id, id_face, mensaje,hilo):        
    cont=0
    content=''
    type_resp=''    
    entidad= {}
    if "postback" in mensaje[0]:
        content=mensaje[0]["postback"]["payload"]
        type_resp="postback"
        #entidad={mensaje[0]["postback"]["payload"]}
    elif "message" in mensaje[0]:
        print(mensaje)
        if "text" in mensaje[0]["message"]:
            content=mensaje[0]["message"]["text"]
        else:
            content=mensaje[0]["message"]["attachments"]
        type_resp="message"    
    estado = "none"
    tipo="none"
    opciones="none"     
    n =  {"payload": {"robot":" ","id_face":" ","recipient_id":" ","access_token":" ","content":" ","type_resp":" ","state":" ", "blocktype":" ", "opciones":" ", "entities":""}}
    data=json.dumps(n)
    decoded=json.loads(data)
    decoded["payload"]["robot"]="none"
    decoded["payload"]["id_face"]=id_face
    decoded["payload"]["access_token"]="none"
    decoded["payload"]["recipient_id"]=recipient_id
    decoded["payload"]["content"]=content
    decoded["payload"]["type_resp"]=type_resp
    decoded["payload"]["state"]=estado
    decoded["payload"]["blocktype"]=tipo
    decoded["payload"]["opciones"]=opciones
    decoded["payload"]["entities"]= entidad
    data_string=json.dumps(decoded)    
    hilo.corr_id = str(uuid.uuid4())
    hilo.channel.basic_consume(on_message_callback=hilo.on_response, auto_ack=True, queue=hilo.queue_name) 
    try:
        hilo.channel.basic_publish(exchange='',
                              routing_key='Colaprueba23',
                              properties=pika.BasicProperties(
                                        reply_to = hilo.queue_name,
                                        correlation_id = hilo.corr_id,
                                        ),
                                body=str(data_string))
        hilo.start()        
        print('\n\n______________________________________________no read__________________________________________________________________________')
        print('\t\tMensaje enviado al Nucleo Logico->\n ',data_string)        
    except Exception as error:
        print("mensaje no enviado...",str(error))

In [6]:
def send_message(payload,access_token):
    api_version = 6.0
    graph_url = 'https://graph.facebook.com/v{0}'.format(api_version)
    request_endpoint = '{0}/me/messages'.format(graph_url)
    auth = {'access_token': access_token}  
    
    response = requests.post(
        request_endpoint,
        params=auth,
        json=payload
    )
    try:
        result = response.json()
        print("\n\n---------------------------------------------------------------------------")
        print("mensaje enviado a la API de facebook:\n",payload)
        return result
    except:
        print("Error al enviar mensaje")
        return 'fail'


In [7]:
def text_message(recipient_id, response,access_token):
    payload={'message': {'text': response}, 'recipient': {'id': recipient_id}, 'notification_type': 'REGULAR'}
    respuesta=send_message(payload,access_token)
    return "success"

In [8]:
def button_message(recipient_id,content,options,access_token):
    buttons = []
    cont=0
    cadena=("".join(options)).capitalize() 
    if cadena.find("<user_email>") != -1:
        for opc in options:
            if opc == "<user_email>":
                button = dict({"content_type":'user_email'})
            elif opc == "<user_phone_number>": 
                button= dict({"content_type":'user_phone_number'})
            else:
                button = dict({"title":opc, "content_type":'text', "payload":opc})
            buttons.append(button)
            cont=cont+1
            if cont>7:
                break    
        payload={'message': {'text': content, 'quick_replies': buttons}, 'recipient': {'id': recipient_id}, 'notification_type': 'REGULAR'}        
    else:
        for opc in options:
            button = Button(title=opc, type='postback', payload=opc)
            buttons.append(button)
            cont=cont+1
            if cont>2:
                break    
        payload={'message': {'attachment': {'type': 'template', 'payload': {'template_type': 'button', 'text': content, 'buttons': buttons}}}, 'recipient': {'id': recipient_id}, 'notification_type': 'REGULAR'}

    result=send_message(payload,access_token)
    assert type(result) is dict
    assert result.get('message_id') is not None
    assert result.get('recipient_id') is not None

In [9]:
def slide_message(recipient_id,content,access_token):
    buttons =[]
    elements=[]
    element={'title':'', 'image_url':'', 'subtitle':'', 'buttons':[]}    
    cont=0    
    for data in content:         
        buttons =[]
        cont=0 
        for btn in data['buttons']:
            typeBtn = btn["typeButton"]
            if typeBtn == "postback":
                button = Button(title=btn['titleButton'], type='postback', payload=btn['payloadButton'])
            else:
                button = Button(title=btn['titleButton'], type='web_url', url=btn['urlButton'])
            buttons.append(button)
            cont=cont+1
            if cont>2:
                break     
        
        element={'title':data['title'], 'image_url':data['image_url'], 'subtitle':data['subtitle'], 'buttons':buttons}
        elements.append(element)   
        
    payload={"recipient": {"id":recipient_id}, "message":{"attachment": {"type":"template", "payload": {"template_type":"generic", "elements":elements}}}, 'notification_type': 'REGULAR'}
    print("pyload slide: ",payload)
    result=send_message(payload,access_token)
    assert type(result) is dict
    assert result.get('message_id') is not None
    assert result.get('recipient_id') is not None

In [10]:
def ticket_message(recipient_id,content,access_token):
    payload={"recipient":{"id":recipient_id},"message":{"attachment":{"type":"template","payload":content}}}    
    result=send_message(payload,access_token)
    assert type(result) is dict
    assert result.get('message_id') is not None
    assert result.get('recipient_id') is not None

In [11]:
def conv(mensaje):
    ans=' '
    contenido=mensaje["Respuesta"]["content"]
    estado=mensaje["Respuesta"]["payload"]["state"]
    entidad= mensaje["Respuesta"]["payload"]["entities"]
    tipo= mensaje["Respuesta"]["blockType"]
    recipient_id=mensaje['Respuesta']['payload']['recipient_id']
    access_token=mensaje['Respuesta']['payload']['access_token']    
    if(tipo=="quickReply" or tipo=="quickReplyDinamico"):
        button_message(recipient_id, contenido, mensaje["Respuesta"]["options"],access_token)
        return entidad, ans, estado, tipo, mensaje["Respuesta"]["options"] 
    elif(tipo=="input" or tipo=="inputDinamico"):
        text_message(recipient_id, contenido,access_token)
        return entidad, ans, estado, tipo, 'vacio'    
    elif(tipo=="informativo" or tipo=="informativoDinamico"):
        text_message(recipient_id, contenido,access_token)
        return entidad, str('next'), estado, tipo, 'vacio'    
    elif(tipo=="slide" or tipo=="slideBuy" or tipo=="slideDinamico"):
        slide_message(recipient_id, contenido,access_token)
        return entidad, str('next'), estado, tipo, 'vacio'
    elif(tipo=="ticket"):
        ticket_message(recipient_id, contenido,access_token)
        return entidad, str('next'), estado, tipo, 'vacio'
    else:
        return entidad, str('hey'), estado, tipo, 'vacio'

In [12]:
class Nucleo_Cliente2(threading.Thread):
    
    def on_response(self, ch, method, props, body):
        if self.corr_id == props.correlation_id:
            self.response = body
            
    def __init__(self, nombreHilo):    
        threading.Thread.__init__(self, name=nombreHilo, target=Nucleo_Cliente2.run)
        self.nombrehilo=nombreHilo
        self.connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
        self.channel = self.connection.channel()
        self.result = self.channel.queue_declare(queue='',exclusive=True)
        self.queue_name = self.result.method.queue
        self.corr_id = ' ' 
        
    def run(self):
        try:                
            self.response = None
            self.response = self.call()
            print(" [.] Got %r", self.response) 
        except Exception as error:
            print("Error run :",str(error))     
            
    def call(self):
        print("hilo consume")
        cont = 0        
        while(cont>=0):
            self.response = None
            while self.response is None:
                self.connection.process_data_events()            
            if len(self.response)>10:
                #print(self.response)
                data = json.loads(self.response)                
                if  data.get("payload"):
                    break
                else:                
                    print('\n_________________________________________________________________________________________________________________________')
                    print('\t\t\nRespuesta del Nucleo Logico-> ',data) 
                    entidad, resp, estado, tipo, opciones= conv(data)
                    break
                


In [ ]:

#-------------Creación de la REST API con Flask-------------
app = Flask(__name__)
#---------------PASO 1. Recepción de mensajes enviados por la API de Facebook.--------------------------
@app.route("/apiREST/v1", methods=['GET', 'POST'])
def receive_message():
    peticiones = 0
    #---------------PASO 2. Inicio: Identificar el tipo de petición que se está solicitando.--------------------------
    if request.method == 'GET':
        print(" petición GET")
        print("metodo GET...")
        print("request",request)
        token_sent = request.args.get("hub.verify_token")
        print(token_sent )
        #---------------PASO 2.1. Llamada a la función que valida la cadena de verificación y se retorna a la petición GET la validación de la función.--------------------------
        return verify_fb_token(token_sent,request.args.get("hub.challenge"))
    else:
        #---------------Recepción de mensajes de una operacion tipo POST.--------------------------        
        
        content = request.get_json(force=True)
        output = request.get_json()
        print('\n_________________________________________________________________________________________________________________________')
        print("Mesaje de la API de Facebook-> \n",output)    
        #return "Message Processed"  
        if output["object"]=="page":
            for event in output['entry']:
                messaging = event['messaging']
                id_face = event['id']
                for message in messaging:
                    nombreHilo="Hilo_"+str(peticiones)
                    N_C = Nucleo_Cliente2(nombreHilo)
                    recipient_id = message['sender']['id']
                    peticiones=peticiones+1                   
                    if message.get('read'):
                        enviarNucleoLectura(recipient_id, id_face, messaging,N_C)                                       
                    elif message.get('message') or message.get('postback'): 
                        enviarNucleo(recipient_id, id_face, messaging,N_C)                
                    #envio de mensaje proveniente de fb a nucleo lógico del sistema
    
    return "Message Processed"        
#-------------Iniciando la REST API-------------
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Jun/2021 19:47:44] "POST /apiREST/v1 HTTP/1.1" 200 -



_________________________________________________________________________________________________________________________
Mesaje de la API de Facebook-> 
 {'object': 'page', 'entry': [{'id': '100750202253729', 'time': 1623718069523, 'messaging': [{'sender': {'id': '4052324791522481'}, 'recipient': {'id': '100750202253729'}, 'timestamp': 1623718069881, 'read': {'watermark': 1623718063408}}]}]}
hilo consume
 [.] Got %r None


127.0.0.1 - - [14/Jun/2021 19:47:44] "POST /apiREST/v1 HTTP/1.1" 200 -



_________________________________________________________________________________________________________________________
Mesaje de la API de Facebook-> 
 {'object': 'page', 'entry': [{'id': '100750202253729', 'time': 1623718069953, 'messaging': [{'sender': {'id': '4052324791522481'}, 'recipient': {'id': '100750202253729'}, 'timestamp': 1623718069833, 'message': {'mid': 'm_z0l7JQ_otkoFwNqxVdlQbHLkAfautiS3NPlmyqjtHIPZgKJn_9QUxyDf97wf7gT9lryIaxChwX71TuiLIfHlCw', 'text': 'Hola'}}]}]}
[{'sender': {'id': '4052324791522481'}, 'recipient': {'id': '100750202253729'}, 'timestamp': 1623718069833, 'message': {'mid': 'm_z0l7JQ_otkoFwNqxVdlQbHLkAfautiS3NPlmyqjtHIPZgKJn_9QUxyDf97wf7gT9lryIaxChwX71TuiLIfHlCw', 'text': 'Hola'}}]
hilo consume

______________________________________________no read__________________________________________________________________________

		Mensaje enviado al Nucleo Logico->
  {"payload": {"robot": "none", "id_face": "100750202253729", "recipient_id": "4052324791522481"

...

 {'object': 'page', 'entry': [{'id': '110373671266869', 'time': 1623733950136, 'messaging': [{'sender': {'id': '110373671266869'}, 'recipient': {'id': '4258311610855241'}, 'timestamp': 1623733950076, 'message': {'mid': 'm_U-9pHyyUh63jIizBxPSoqfTGI33Qv4BHTBl73koF-z3jNXqepTpKVGsKL0uB-OK94NzSbWMpLCCBjCfRiIASuw', 'is_echo': True, 'text': 'Gracias por información... Adios!', 'app_id': 843376636183033}}]}]}

_________________________________________________________________________________________________________________________
Mesaje de la API de Facebook-> 
 {'object': 'page', 'entry': [{'id': '110373671266869', 'time': 1623733950180, 'messaging': [{'sender': {'id': '4258311610855241'}, 'recipient': {'id': '110373671266869'}, 'timestamp': 1623733950227, 'delivery': {'mids': ['m_U-9pHyyUh63jIizBxPSoqfTGI33Qv4BHTBl73koF-z3jNXqepTpKVGsKL0uB-OK94NzSbWMpLCCBjCfRiIASuw'], 'watermark': 1623733950076}}]}]}

___________________________________________________________________________________________

127.0.0.1 - - [15/Jun/2021 00:12:24] "POST /apiREST/v1 HTTP/1.1" 200 -



_________________________________________________________________________________________________________________________
Mesaje de la API de Facebook-> 
 {'object': 'page', 'entry': [{'id': '110373671266869', 'time': 1623733950485, 'messaging': [{'sender': {'id': '4258311610855241'}, 'recipient': {'id': '110373671266869'}, 'timestamp': 1623733950534, 'delivery': {'mids': ['m_U-9pHyyUh63jIizBxPSoqfTGI33Qv4BHTBl73koF-z3jNXqepTpKVGsKL0uB-OK94NzSbWMpLCCBjCfRiIASuw'], 'watermark': 1623733950076}}]}]}

_________________________________________________________________________________________________________________________
		
Respuesta del Nucleo Logico->  {'Respuesta': {'content': 'Soy el bot 2021', 'next_id': 'Informativo | 98a8f39d-6761-48e7-aca7-66279cfabf40 | 96628859-0899-7537-4503-489244348545 | ZG7udwy8rzhE5xtUnOGl', 'blockType': 'informativo', 'contentType': 'text', 'typingTime': '1', 'payload': {'content': '', 'state': 'Informativo | a0be25bb-4eb4-49c2-830f-6eb23d64966c | ZG7udwy8

127.0.0.1 - - [15/Jun/2021 00:12:25] "POST /apiREST/v1 HTTP/1.1" 200 -



_________________________________________________________________________________________________________________________
Mesaje de la API de Facebook-> 
 {'object': 'page', 'entry': [{'id': '110373671266869', 'time': 1623733951426, 'messaging': [{'sender': {'id': '4258311610855241'}, 'recipient': {'id': '110373671266869'}, 'timestamp': 1623733951540, 'read': {'watermark': 1623733951066}}]}]}
hilo consume
